In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_ibd = pd.read_csv("../../ag3_results/ibd/AG1000G-GH/ibd.tsv", sep="\t")

In [ ]:
df_ibd = df_ibd.assign(cm=lambda x: x.lengthM * 100, 
             size=lambda x: x.EndBP - x.StartBP,
                size_snps=lambda x: x.End - x.Start).assign(cm_snp_ratio=lambda x: x.size_snps / x.cm,
                midpoint=lambda x: (x.EndBP + x.StartBP)/2)

In [ ]:
import malariagen_data
ag3 = malariagen_data.Ag3()
total_genome_size = np.sum([ag3.genome_sequence(c).shape[0] for c in ('2RL', '3RL')])

In [ ]:
px.histogram(df_ibd['cm_snp_ratio'])

In [ ]:
px.scatter(df_ibd.query("cm < 100"), x='size_snps', y='cm', hover_data=['StartBP', 'EndBP', 'ch'], color='cm_snp_ratio', template='simple_white')

In [ ]:
gff = ag3.genome_features('2RL').query("type == 'gene'")
gff.query("ID in ['AGAP004707', 'AGAP006028']")

In [ ]:
px.histogram(df_ibd.query("ch == '2RL'").midpoint, template='simple_white')

In [ ]:
px.histogram(df_ibd.query("ch == '3RL'").midpoint, template='simple_white')

In [ ]:
df_samples = ag3.sample_metadata("1244-VO-GH-YAWSON-VMF00149")

In [ ]:
df_samples = df_samples.iloc[:, :2]

In [ ]:
result = df_ibd.groupby(['iid1', 'iid2']).agg({
    'size': ['count', 'mean', 'sum'],
    'cm': ['mean', 'sum']
})

# Rename the columns for clarity
result.columns = ['count', 'size_mean', 'size_total', 'cm_mean', 'cm_total']

# Reset the index to make 'var1' and 'var2' regular columns
result = result.reset_index()
result = result.assign(fraction_genome_ibd=lambda x: x.size_total / total_genome_size)
result = result.assign(kinship_coeff = lambda x: x.fraction_genome_ibd / 2)   # need editing 

result = result.merge(df_samples, left_on='iid1', right_on='sample_id').merge(df_samples, left_on='iid2', right_on='sample_id').drop(columns=['sample_id_x', 'sample_id_y'])

result.sort_values('kinship_coeff', ascending=False)[:50]

In [ ]:
import plotly.express as px

In [ ]:
px.histogram(result['kinship_coeff'], template='simple_white')